In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

%pylab inline

In [ ]:
input = cv2.imread('/content/drive/MyDrive/texture.png')

In [ ]:
cv2_imshow(input)


Начнём с простого повторения изображения по схеме 3х3 без изменения ориентации тайлов и применения каких-либо методов обработки, чтобы оценить проблемы, которые предстоит решать.

In [ ]:
def texture_repeat(input):
    duplicated = cv2.repeat(input, 3, 3)
    return duplicated

In [ ]:
duplicated = texture_repeat(input)
cv2_imshow(duplicated)


Текстура очевидно не является "бесшовной"

Отчётливо видны повторяющиеся паттерны оригинальной текстуры, цветовые переходы и стыки на границах тайлов.

Особенно выделяются более тёмный верхний левый угол каждого тайла, вертикальные стыки между тайлами из-за переходв цвета, и горизонтальная линия сгиба ткани по центру.

# 1. Метод отражений
Попробуем метод отражений: в центре помещается исходный тайл, сверху и снизу от него - исходный тайл, отражённый по вертикали, слева и справа - исходный тайл, отражённый по горизонтали, по углам - исходный тайл, отражённый по вертикали и по горизонтали.   

In [ ]:
def texture_mirror(input):
    input_flipped_horz = cv2.flip(input, 1)
    input_flipped_vert = cv2.flip(input, 0)
    input_flipped_both = cv2.flip(input, -1)
    top = np.hstack([input_flipped_both, input_flipped_vert, input_flipped_both])
    mid = np.hstack([input_flipped_horz, input, input_flipped_horz])
    bot = np.hstack([input_flipped_both, input_flipped_vert, input_flipped_both])
    result_mirror = np.vstack([top, mid, bot])
    return result_mirror

In [ ]:
result_mirror = texture_mirror(input)
cv2_imshow(result_mirror)

Заметных цветовых переходов на границах тайлов больше нет, однако границы видны по образовавшимся зеркальным артефактам.
Присутствуют повторяющиеся паттерны оригинальной текстуры.

# 2. Альфа-смешивание

Попробуем убрать швы между тайлами путём смешивания пограничных участков



In [ ]:
# Размеры изображения
h, w, c = input.shape

In [ ]:
# Ширина обрабатываемых краёв в процентах от размера изображения (ширины и высоты, соответственно)
BORDER_WIDTH_PERCENT_VERT = 6
BORDER_WIDTH_PERCENT_HORZ = 5
# Определяем центральные зоны для сглаживания швов
border_size_vert = int(BORDER_WIDTH_PERCENT_VERT / 100 * w)
border_size_horz = int(BORDER_WIDTH_PERCENT_HORZ / 100 * h)

In [ ]:
# Начнём с вертикальных швов
# Сначала создадим 2 новых тайла, сдвинутых по горизонтали
shifted_horz_left = np.roll(input, shift= w // 2 + border_size_vert, axis=1)
shifted_horz_right = np.roll(input, shift= w // 2 - border_size_vert, axis=1)

In [ ]:
# Создаём градиентную альфа-маску для плавного перехода
horiz_gradient = np.tile(np.linspace(0.0, 1.0, 2 * border_size_vert).reshape(1, -1, 1), (h, 1, 3))

alpha_mask_horz = np.zeros((h, w, 3))

alpha_mask_horz[:, w // 2 - border_size_vert: w // 2 + border_size_vert, :] = horiz_gradient
alpha_mask_horz[:, w // 2 + border_size_vert:, :] = 1

# Обратная маска
alpha_mask_horz_reverted = 1 - alpha_mask_horz

In [ ]:
# Применяем маску для получения правой части
half_right = cv2.multiply(alpha_mask_horz, shifted_horz_right.astype(np.float32), dtype=cv2.CV_32F)

# Применяем обратную маску для получения левой части
half_left = cv2.multiply(alpha_mask_horz_reverted, shifted_horz_left.astype(np.float32), dtype=cv2.CV_32F)

# Суммируем полученные части
outImage_vert_blend = cv2.add(half_right, half_left, dtype=cv2.CV_32F)

In [ ]:
cv2_imshow(outImage_vert_blend)

In [ ]:
# Сделаем то же самое для горизонтальных швов
# Убираем горизонтальные швы
shifted_vert_top = np.roll(outImage_vert_blend, shift= h // 2 + border_size_horz, axis=0)
shifted_vert_bottom = np.roll(outImage_vert_blend, shift= h // 2 - border_size_horz, axis=0)

#
vert_gradient = np.tile(np.linspace(0.0, 1.0, 2 * border_size_horz).reshape(-1, 1, 1), (1, w, 3))

alpha_mask_vert = np.zeros((h, w, 3))
#
alpha_mask_vert[h // 2 - border_size_horz : h // 2 + border_size_horz, :, :] = vert_gradient
alpha_mask_vert[h // 2 + border_size_horz :, :, :] = 1

alpha_mask_vert_reverted = 1 - alpha_mask_vert

In [ ]:
# Применяем обратную маску для получения верхней части
half_top = cv2.multiply(alpha_mask_vert_reverted, shifted_vert_top.astype(np.float32), dtype=cv2.CV_32F)

# Применяем маску для получения нижней части
half_bottom = cv2.multiply(alpha_mask_vert, shifted_vert_bottom.astype(np.float32), dtype=cv2.CV_32F)

# Суммируем полученные части
outImage_blend = cv2.add(half_top, half_bottom, dtype=cv2.CV_32F)

In [ ]:
# Возвращаем тайл в изначальное полжение
original_tile_processed = np.roll(outImage_blend, shift= (h // 2, w // 2), axis = (0, 1))

In [ ]:
output_texture_repeat = texture_repeat(original_tile_processed)
cv2_imshow(output_texture_repeat)

Таким образом, мы избавились от заметных цветовых переходов и стыков на границах тайлов. Остались видны только горизонтальные линии сгиба ткани, присутствующие и на оригинальном изображении. При необходимости можно сгладить их тем же методом альфа-блендинга.

Основные минусы решения:
1. Решение адаптировано под конкретную текстуру ткани с мелким плетением. В случае наличия более крупных различимых элементов метод альфа-блендинга не подойдёт
2. Обработанные стыки могут выглядеть менее резкими, чем остальное изображение.

В качестве дальнейших шагов можно попробовать использовать фильтр GaussianBlur, чтобы выделить размытую составляющую и вычесть её из изображения. Однако применение фильтров ко всему изображению повлияет на резкость исходного тайла, а применеие фильтров только к обработанным участкам может сделать их ещё более заметными.

Чтобы сгладить различия в резкости, также можно попробовать использовать фильтр размытия на всём изображении, а затем одну из моделей для решения задачи SuperResolution. В случае с исходной текстурой данный подход кажется неоправданно сложным.

Также можно использовать Stable Diffusion Inpainting для генерирования частей изображения в области стыков и создания бесшовной текстуры

In [ ]:
blurred_image = cv2.GaussianBlur(output_texture_repeat, (0,0), 3)
sharpened_image = cv2.addWeighted(output_texture_repeat, 1.5, blurred_image, -0.5, 0)

In [ ]:
cv2_imshow(sharpened_image)

# Конец предложенного решения

#  3. Готовые open-source решения
  Существуют также готовые решения для похожей задачи создания бесшовных текстур:


1.   https://github.com/rtmigo/img2texture/tree/master - использует альфа-блендинг
2.   https://github.com/sagieppel/convert-image-into-seamless-tileable-texture - более сложный подход, учитывающий разницу между краями изображения при смешивании и использующий Gaussian blur для сглаживания переходов.
3. https://github.com/sagieppel/transform-image-into-seamless-tileable-texture-using-stable-diffusion-inpainting - использование StableDiffusion для преобразования краёв тайла с целью создания бесшовной текстуры
4. https://github.com/TheRealMAV/texture-synthesis/tree/main - генерирование новой текстуры желаемого размера на основе исходного тайла (случайным образом выбирается блок из исходного изображения, далее полсдеовательно подбираются наиболее подходящие блоки для вставки справа и снизу от выбранного блока)  



Ниже только тестирование двух из перечисленный готовых решений на предоставленной текстуре для сравнения с полученными выше результатами.


Метод 1

Следующий метод также основан на подходе альфа-блендинга, но он обрезает края изображения. Поэтому он нам не подходит, используется для сравнения результатов

Источник:
https://github.com/rtmigo/img2texture/tree/master

In [ ]:
!pip install img2texture

In [ ]:
from img2texture import image_to_seamless

In [ ]:
# load PIL image
src_image = Image.open('/content/drive/MyDrive/texture.png')

# convert to seamless PIL image
result_image = image_to_seamless(src_image, overlap=0.1)

In [ ]:
result_image

In [ ]:
width, height = result_image.size

In [ ]:
# Создаём новое изображение 3×3
result = Image.new("RGB", (width * 3, height * 3))

# Раскладываем изображения в сетку
# Центр — оригинал
result.paste(result_image, (0, 0))             # левый верх
result.paste(result_image, (width, 0))              # верхний центр
result.paste(result_image, (2 * width, 0))         # правый верх

result.paste(result_image, (0, height))              # левый центр
result.paste(result_image, (width, height))                 # центр — оригинал
result.paste(result_image, (2 * width, height))          # правый центр

result.paste(result_image, (0, 2 * height))         # левый низ
result.paste(result_image, (width, 2 * height))          # нижний центр
result.paste(result_image, (2 * width, 2 * height))     # правый низ

In [ ]:
result

Метод 4

Источник:
https://github.com/TheRealMAV/texture-synthesis/tree/main

In [ ]:
src = cv2.imread('/content/drive/MyDrive/texture.png')
src = src.astype(np.float32)


# parameters :
randomness = 15
divide = 4

height, width, _ = src.shape
bl = int(width / divide)  # block length

result = np.zeros([3960, 3960, 3])

In [ ]:
init_y = np.random.randint(height - bl - 1)
init_x = np.random.randint(width - bl - 1)

block = np.copy(src[init_y:init_y + bl, init_x:init_x + bl, :])
result[0:bl, 0:bl, :] = np.copy(block)

result = result.astype(np.float32)

In [ ]:
cv2_imshow(block)

(Сгенерированное описание)
Выбранный блок кода определяет функцию getNextBlockRow. Эта функция предназначена для поиска подходящего блока из исходного изображения (src) для размещения справа от заданного блока и смешивания их для создания плавного горизонтального перехода.

Вот краткое описание того, что делает код:

1. Извлекает полосу из текущего блока: берет узкую вертикальную полосу с правой стороны входного блока. Эта полоса представляет собой область, которая будет перекрываться со следующим блоком.
2. Вычисляет SSD (Сумму квадратов различий): Вычисляет сумму квадратов различий между этой полосой и потенциальными соответствующими полосками в блоках исходного изображения (src). Это помогает находить блоки в исходном изображении, левый край которых похож на правый край текущего блока.
3. Выбирает случайный похожий блок: из случайного числа наиболее подходящих блоков (с наименьшим количеством SSD) случайным образом выбирается один. Это добавляет вариативности результирующей текстуре.
4. Извлекает соответствующую полосу из следующего блока: берет вертикальную полосу с левой стороны выбранного следующего блока.
5. Вычисляет разницу между полосами: Вычисляет разницу между полосой из текущего блока и полосой из следующего блока.
6. Находит путь с минимальными затратами: Вычисляет матрицу "затрат" на основе различий между двумя полосами. Затем он находит путь по этой матрице затрат, который сводит к минимуму суммарную разницу, эффективно находя наименее заметную линию шва для горизонтального соединения.
7. Совмещение по траектории минимальных затрат: Две полосы соединяются по рассчитанной траектории минимальных затрат. Это создает более плавный горизонтальный переход, чем простой жесткий разрез.
8. Интегрирует смешанную полосу в следующий блок: она заменяет исходную левую полосу следующего блока на новую смешанную полосу.
9. Возвращает смешанный блок: Функция возвращает следующий блок с плавно смешанным левым краем.


In [ ]:
def getNextBlockRow(block, src):

    strip = np.copy(block[:, int(bl / 5 * 4):, :])
    strip_h, strip_w, _ = strip.shape

    ssd = np.zeros([height, width])

    for i in range(3):
        ker = np.ones(strip[:, :, i].shape)
        C = np.sum(strip[:, :, i] * strip[:, :, i])
        ssd += cv2.filter2D(src[:, :, i] * src[:, :, i], -1, ker) - 2 * (
            cv2.filter2D(src[:, :, i], -1, strip[:, :, i])) + C

    ssd = ssd[int(strip_h / 2):height - bl, int(strip_w / 2):width - bl]

    ssd_flat = ssd.flatten()

    k = randomness

    random_index = np.argpartition(ssd_flat, k)[np.random.randint(k)]

    x = int(random_index % ssd.shape[1])
    y = int(random_index / ssd.shape[1])

    next_block = np.copy(src[y:y + bl, x:x + bl, :])

    next_strip = np.copy(next_block[0:strip_h, 0:strip_w, :])

    strip_diff = strip - next_strip
    strip_diff *= strip_diff

    strip_diff = cv2.cvtColor(strip_diff, cv2.COLOR_BGR2GRAY)

    cost = np.copy(strip_diff)
    for i in range(1, strip_h):
        for j in range(strip_w):
            if j == 0:
                cost[i, j] += min([cost[i - 1, j], cost[i - 1, j + 1]])
            elif j == strip_w - 1:
                cost[i, j] += min([cost[i - 1, j], cost[i - 1, j - 1]])
            else:
                cost[i, j] += min([min([cost[i - 1, j], cost[i - 1, j - 1]]), cost[i - 1, j + 1]])

    res_strip = np.zeros(strip.shape)

    line_index = np.argmin(cost[strip_h - 1])
    for i in range(strip_h - 1, -1, -1):

        res_strip[i, :line_index, :] = strip[i, :line_index, :]
        res_strip[i, line_index:, :] = next_strip[i, line_index:, :]
        if line_index == 0:
            line_index += np.argmin((cost[i])[line_index:line_index + 2])
        elif line_index == strip_w - 1:
            line_index += np.argmin((cost[i])[line_index - 1:line_index + 1]) - 1
        else:
            line_index += np.argmin((cost[i])[line_index - 1:line_index + 2]) - 1

    next_block[:, 0:strip_w, :] = res_strip
    return next_block


(Сгенерированное описание)
Выбранный блок кода определяет функцию getNextBlockCol. Эта функция похожа на getNextBlockRow, но предназначена для поиска подходящего блока из исходного изображения (src), размещения его под заданным блоком и смешивания их для создания плавного вертикального перехода.

Вот краткое описание того, что делает код:

1. Извлекает полосу из текущего блока: берет узкую горизонтальную полосу с нижней стороны входного блока. Эта полоса представляет собой область, которая будет перекрываться с блоком, расположенным под ней.
2. Вычисляет SSD (Сумму квадратов различий): Вычисляет сумму квадратов различий между этой полосой и потенциальными соответствующими полосами в блоках исходного изображения (src). Это помогает найти блоки в исходном изображении, верхний край которых похож на нижний край текущего блока.
3. Выбирает случайный похожий блок: из случайного числа наиболее подходящих блоков (с наименьшим SSD) случайным образом выбирается один. Это добавляет вариации к результирующей текстуре.
4. Извлекает соответствующую полосу из следующего блока: берет горизонтальную полосу с верхней стороны выбранного следующего блока.
5. Вычисляет разницу между полосами: Вычисляет разницу между полосой из текущего блока и полосой из следующего блока.
6. Находит путь с минимальными затратами: вычисляет матрицу затрат на основе различий между двумя полосами. Затем находит путь через эту матрицу затрат, который минимизирует суммарную разницу, эффективно находя наименее заметную линию шва для вертикального соединения.
7. Смешивание по траектории с минимальными затратами: Две полосы смешиваются по рассчитанной траектории с минимальными затратами. Это создает более плавный вертикальный переход, чем простой жесткий разрез.
8. Объедините смешанную полосу в следующем блоке: замените исходную верхнюю полосу следующего блока новой смешанной полосой.
9. Возвращает смешанный блок: Функция возвращает следующий блок с плавно смешанным верхним краем.


In [ ]:
def getNextBlockCol(block, src):

    strip = np.copy(block[int(bl / 5 * 4):, :, :])
    strip_h, strip_w, _ = strip.shape

    ssd = np.zeros([height, width])

    for i in range(3):
        ker = np.ones(strip[:, :, i].shape)
        C = np.sum(strip[:, :, i] * strip[:, :, i])
        ssd += cv2.filter2D(src[:, :, i] * src[:, :, i], -1, ker) - 2 * (
            cv2.filter2D(src[:, :, i], -1, strip[:, :, i])) + C

    ssd = ssd[int(strip_h / 2):height - bl, int(strip_w / 2):width - bl]

    ssd_flat = ssd.flatten()

    k = randomness

    random_index = np.argpartition(ssd_flat, k)[np.random.randint(k)]

    x = int(random_index % ssd.shape[1])
    y = int(random_index / ssd.shape[1])

    next_block = np.copy(src[y:y + bl, x:x + bl, :])

    next_strip = np.copy(next_block[0:strip_h, 0:strip_w, :])

    strip_diff = strip - next_strip
    strip_diff *= strip_diff

    strip_diff = cv2.cvtColor(strip_diff, cv2.COLOR_BGR2GRAY)

    cost = np.copy(strip_diff)
    for j in range(1, strip_w):
        for i in range(strip_h):
            if i == 0:
                cost[i, j] += min([cost[i, j - 1], cost[i + 1, j - 1]])
            elif i == strip_h - 1:
                cost[i, j] += min([cost[i, j - 1], cost[i - 1, j - 1]])
            else:
                cost[i, j] += min([min([cost[i, j - 1], cost[i - 1, j - 1]]), cost[i + 1, j - 1]])

    res_strip = np.zeros(strip.shape)

    line_index = np.argmin(cost[:, strip_w - 1])

    for j in range(strip_w - 1, -1, -1):

        res_strip[:line_index, j, :] = strip[:line_index, j, :]
        res_strip[line_index:, j, :] = next_strip[line_index:, j, :]
        if line_index == 0:
            line_index += np.argmin((cost[:, i])[line_index:line_index + 2])
        elif line_index == strip_h - 1:
            line_index += np.argmin((cost[:, j])[line_index - 1:line_index + 1]) - 1
        else:
            line_index += np.argmin((cost[:, j])[line_index - 1:line_index + 2]) - 1

    next_block[0:strip_h, :, :] = res_strip
    return next_block

(Сгенерированное описание)
Эта функция предназначена для поиска подходящего блока из исходного изображения для размещения в средних частях увеличенного результирующего изображения, смешивания его с существующими окружающими блоками для создания плавного перехода.

Вот краткое описание того, что делает код:

1.   Вычисляет SSD (Сумму квадратов различий): он вычисляет сумму квадратов различий между текущим блоком и потенциальными следующими блоками исходного изображения в пределах определенной области (исключая области наложения). Это помогает находить блоки-кандидаты, которые визуально похожи.
2.   Выбирает случайный похожий блок: из случайного числа наиболее подходящих блоков (с наименьшим SSD) случайным образом выбирается один. Это вносит разнообразие и позволяет избежать простого повторения одного и того же блока.
3.   Вычисляет разностные полосы: вычисляет разницу между текущим блоком и выбранным следующим блоком в областях перекрытия (вертикальные и горизонтальные полосы, где они будут соединены).
4.   Находит путь с минимальными затратами: как для областей вертикального, так и для горизонтального перекрытия программа вычисляет матрицу затрат на основе различий. Затем программа находит путь через эту матрицу затрат, который минимизирует суммарную разницу, эффективно находя наименее заметную линию шва.
5.   Совмещение по траектории с минимальными затратами: Текущий блок и выбранный следующий блок совмещаются по траекториям с минимальными затратами, рассчитанным как в вертикальной, так и в горизонтальной областях перекрытия.
6.   Это создает более плавный переход, чем простое резкое вырезание.
Возвращает смешанный блок: Функция возвращает следующий блок со смешанными краями.

In [ ]:
def getNextBlockMid(block, src):

    ssd = np.zeros([height, width])

    ker = np.ones([bl, bl])
    ker[int(bl / 5):, int(bl / 5):] = 0

    for i in range(3):
        C = np.sum(block[:, :, i] * block[:, :, i])
        ssd += cv2.filter2D(src[:, :, i] * src[:, :, i], -1, ker) - 2 * (
            cv2.filter2D(src[:, :, i], -1, block[:, :, i])) + C

    ssd = ssd[int(bl / 2):height - bl, int(bl / 2):width - bl]

    ssd_flat = ssd.flatten()

    k = randomness

    random_index = np.argpartition(ssd_flat, k)[np.random.randint(k)]

    x = int(random_index % ssd.shape[1])
    y = int(random_index / ssd.shape[1])

    next_block = np.copy(src[y:y + bl, x:x + bl, :])

    block_diff = block - next_block
    block_diff *= block_diff
    block_diff = cv2.cvtColor(block_diff, cv2.COLOR_BGR2GRAY)

    ver_diff = np.copy(block_diff[:, :int(bl / 5)])
    hor_diff = np.copy(block_diff[:int(bl / 5), :])

    cost = np.copy(ver_diff)
    for i in range(1, bl):
        for j in range(int(bl / 5)):
            if j == 0:
                cost[i, j] += min([cost[i - 1, j], cost[i - 1, j + 1]])
            elif j == int(bl / 5) - 1:
                cost[i, j] += min([cost[i - 1, j], cost[i - 1, j - 1]])
            else:
                cost[i, j] += min([min([cost[i - 1, j], cost[i - 1, j - 1]]), cost[i - 1, j + 1]])

    res_block = np.copy(next_block)

    line_index = np.argmin(cost[bl - 1])
    for i in range(bl - 1, -1, -1):

        res_block[i, :line_index, :] = block[i, :line_index, :]

        if line_index == 0:
            line_index += np.argmin((cost[i])[line_index:line_index + 2])
        elif line_index == int(bl / 5) - 1:
            line_index += np.argmin((cost[i])[line_index - 1:line_index + 1]) - 1
        else:
            line_index += np.argmin((cost[i])[line_index - 1:line_index + 2]) - 1

    cost = np.copy(hor_diff)
    for j in range(1, bl):
        for i in range(int(bl / 5)):
            if i == 0:
                cost[i, j] += min([cost[i, j - 1], cost[i + 1, j - 1]])
            elif i == int(bl / 5) - 1:
                cost[i, j] += min([cost[i, j - 1], cost[i - 1, j - 1]])
            else:
                cost[i, j] += min([min([cost[i, j - 1], cost[i - 1, j - 1]]), cost[i + 1, j - 1]])

    line_index = np.argmin(cost[:, bl - 1])

    for j in range(bl - 1, -1, -1):

        res_block[:line_index, j, :] = block[:line_index, j, :]

        if line_index == 0:
            line_index += np.argmin((cost[:, i])[line_index:line_index + 2])
        elif line_index == int(bl / 5) - 1:
            line_index += np.argmin((cost[:, j])[line_index - 1:line_index + 1]) - 1
        else:
            line_index += np.argmin((cost[:, j])[line_index - 1:line_index + 2]) - 1

    return res_block


(Сгенерированное описание)
Выбранный блок кода отвечает за заполнение результирующего изображения (большого чистого холста) блоками, взятыми из исходного изображения (src), с помощью функций getNextBlockRow, getNextBlockCol и getNextBlockMid, которые вы определили ранее.

По шагам:

1. Заполнить первую строку (после начального блока):

Она начинается с начального блока, расположенного в верхнем левом углу результата.
Затем выполняется итерация по горизонтали, вызывается getNextBlockRow, чтобы найти и смешать следующий блок справа.
Каждый смешанный блок помещается в соответствующую позицию в первой строке результирующего изображения.
Цикл продолжается до тех пор, пока первая строка результирующего изображения не будет заполнена по горизонтали.

2. Заполнить первый столбец (после начального блока):

Он возвращает текущий блок к исходному блоку в верхнем левом углу.
Затем выполняется вертикальная итерация с вызовом getNextBlockCol для поиска и наложения следующего блока вниз.
Каждый наложенный блок помещается в соответствующую позицию в первом столбце результирующего изображения.
Цикл продолжается до тех пор, пока первый столбец результирующего изображения не будет заполнен по вертикали.

3. Заполнить средние участки:

Затем используются вложенные циклы для итерации по оставшимся "средним" участкам результирующего изображения (исключая первую строку и столбец, которые уже заполнены).
Для каждой средней части он берет существующий блок из результирующего изображения в этой позиции.
Он вызывает getNextBlockMid, чтобы найти подходящий блок из исходного изображения и смешать его с окружающими блоками, которые уже есть в результате.
Затем смешанный блок помещается обратно в результирующее изображение.

4. Обрезка результата:

Наконец, изображение обрезается до размера 3300x3300 пикселей.
По сути, этот блок кода управляет процессом разбиения исходного изображения на фрагменты путем разумного выбора и смешивания блоков для создания более крупной, кажущейся бесшовной текстуры.

In [ ]:
    for i in range(1, int(np.ceil(3300 / int(bl / 5 * 4)))):
        block = getNextBlockRow(block, src)
        result[0:bl, int(bl / 5 * 4) * i:bl + int(bl / 5 * 4) * i, :] = block

    block = np.copy(result[0:bl, 0:bl, :])
    for i in range(1, int(np.ceil(3300 / int(bl / 5 * 4)))):
        block = getNextBlockCol(block, src)
        result[int(bl / 5 * 4) * i:bl + int(bl / 5 * 4) * i, 0:bl, :] = block

    for i in range(1, int(np.ceil(3300 / int(bl / 5 * 4)))):
        for j in range(1, int(np.ceil(3300 / int(bl / 5 * 4)))):
            block = np.copy(
                result[int(bl / 5 * 4) * i:bl + int(bl / 5 * 4) * i, int(bl / 5 * 4) * j:bl + int(bl / 5 * 4) * j, :])
            result[int(bl / 5 * 4) * i:bl + int(bl / 5 * 4) * i, int(bl / 5 * 4) * j:bl + int(bl / 5 * 4) * j,
            :] = getNextBlockMid(block, src)

    result = result[:3300, :3300, :]


In [ ]:
cv2_imshow(result)

Минусы: очень медленный алгоритм, заимствованный код требует рефакторинга и пояснений, присутствуют blocking artifacts